# ReInvent 2019 Amazon Personalize 研讨会

![主图](static/imgs/image.png)

## 日程

概括来说，使用 Amazon Personalize 的步骤如下图所示：

![流程图](static/imgs/personalize_process.png)

您将遵循的具体流程包括：

1. 导入和设置
1. 准备数据
1. 导入数据
1. 选择配方
1. 训练解决方案
1. 部署活动
1. 获得推荐
1. 实时交互
1. 结论
1. 额外材料：批量导出推荐

在这些步骤中，您将看到并执行使用我们的 Boto3 SDK 在 Python 中编写的代码片段，这些片段可以修改为您的产品与 Personalize 集成的组件。

此笔记本将指导您完成为特定用户量身定制的电影推荐模型的构建步骤，目标是根据用户与电影的积极交互历史来推荐电影。

这些数据是通过 [MovieLens 项目](https://movielens.org)提供的，如果您感兴趣，可以稍后阅读更多相关内容。

以下内容旨在指导您在研讨会的时间线内完成这些流程。您可以稍后将您在 GitHub 上找到的 CloudFormation 模板部署到您自己的账户，并可以浏览笔记本 1-3 以再次执行相同练习。此外，您可以更新单元格中的内容以反映您自己的数据，这是为您的使用案例构建自定义推荐模型的有效途径。

## 如何使用笔记本

代码被分解为如下所示的单元格。在本页顶部有一个三角形 `Run` 按钮，您可以单击该按钮以执行每个单元格，并移至下一个单元格；也可以按 `Shift` + `Enter` 组合键或 `Shift` + `Return` 组合键（Mac 用户），在单元格中执行并移至下一个单元格。


当一个单元格执行时，您会注意到该单元格运行时旁边有一行显示 `*`，或者它会更新为一个数字，以指示在执行完单元格中的所有代码后完成执行的最后一个单元格。


只需按照以下说明执行单元格，即可开始使用 Amazon Personalize。



## 导入和设置 

Python 附带有许多库，我们需要导入这些库以及已安装的库来帮助我们，比如 boto3（AWS SDK）和 Pandas/NumPy，它们都是核心数据科学工具。下面的单元格将导入它们以供在此处使用。它还会将 boto3 库更新至最新版本。您可能会看到警告（黄色文本）或错误（红色文本），这完全没有问题，只需继续运行下一个单元格即可。

In [ ]:
# Update boto3
!pip install --upgrade boto3
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime
import uuid

接下来，您需要验证您的环境是否可以成功地与 Amazon Personalize 进行通信，下面的行就可以实现这一目的。

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

设置的最后一步是确定您运行此研讨会的区域。下面的单元格将执行此操作，并分配给 `region` 变量。

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]

## 准备数据

首先，您需要收集用户以某种方式与内容交互的数据点。Amazon Personalize 假设，如果记录了交互，则它就是积极的，稍后您将看到我们如何使用它。

下面的单元格将下载您需要的数据，从 ZIP 文件中提取内容，然后将其中的一小部分显示到屏幕上。

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'], engine='python')
pd.set_option('display.max_rows', 5)
data

正如您所看到的，数据包含 UserID、ItemID、Rating 和 Timestamp。

我们不需要 `Rating` 列，因此在保存文件之前它将被删除。

数据准备就绪后，您可以将数据保存到本地的 CSV 文件中，并保留最后一行。

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
filename = "movie-lens-ml-100k-prepared.csv"
data.to_csv(filename, index=False)

在将数据上载到 S3 之前，您需要创建一个存储桶，下面的单元格将执行此操作。

In [ ]:
print(region)
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "reinventpersonalizeworkshop"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

现在，您可以将文件上载到 S3，它将准备好导入到 Amazon Personalize。

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

## 导入数据

导入数据的步骤如下：

1. 创建数据集组。
1. 确定数据集的架构。
1. 使用您的架构创建数据集。
1. 运行 ImportJob 以加载用于 Personalize 的数据。


在 Amazon Personalize 中，数据集组是将您的信息与任何其他实验隔离的方式。这些组之间根本不会共享任何信息。数据集组可以包含交互数据、项目元数据、用户元数据、事件跟踪器、解决方案和活动。了解更多信息：https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html 

首先创建一个数据集组

#### 创建数据集组

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-RI-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### 数据集组创建循环

创建数据集组需要几秒钟时间，因此下面的单元格将进行轮询，直到数据集组处于活跃状态，然后您可以继续研讨会。在显示 `ACTIVE` 时，移动到 `Create Schema` 步骤。


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(5)

#### 创建架构


现在，您可以确定数据集的架构。在此研讨会中，由于时间有限，因此您将仅使用用户-项目-交互数据（或简称为交互数据）。下面的单元格包含此数据集的必需项目，并映射到您之前上载的 CSV 的结构。

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

下面的单元格将在 Amazon Personalize 中创建一个可以连接到数据集的架构，这就是 Personalize 理解 CSV 文件中的内容的方式。

In [ ]:
create_schema_response = personalize.create_schema(
    name = "personalize-ri-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

#### 创建数据集


接下来，创建交互数据集，并将提供的架构分配给它，然后将数据集分配给您之前使用以下代码创建的数据集组：

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ri-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### 将策略附加到 S3 存储桶

在为您的数据执行导入作业之前，您需要将一个存储桶策略附加到 S3 存储桶以允许 Personalize 与存储桶通信，还需要一个 IAM 角色以供该服务在此 AWS 账户中使用。

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

请注意，下面单元格中的 IAM 更改只需几秒钟即可得到确认，因此完成此部分需要 1 分钟。接下来执行下面的单元格。

#### 创建 Personalize 角色

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRIDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

最后，您准备将数据导入到 Personalize 中，下面的第一个单元格将启动该过程，第二个单元格包含一个 While 循环，该循环将轮询服务以确定导入作业何时完成。

在本次研讨会期间，您将导入一个相对较小的数据集，对于这样一个小文件，这似乎需要一段时间。在这里，时间的主要用途是提供将实际运行任务的专用资源。例如，这允许 Personalize 提供 HIPAA 合规性，这也意味着，它不会仅仅因为您的文件较大就花费非常长的时间，资源启动并运行之后的导入会非常快。

完成导入过程最多可能需要 20 分钟。 

#### 创建数据集导入作业

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ri-import",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### 数据集导入循环

下面的单元格将需要更长的时间才能完成，它将要轮询以了解您的数据集何时已完全导入到 Personalize 中。这大约需要 20 分钟才能完成。大部分时间都花在幕后预置基础结构上。这意味着在大多数情况下，导入较大文件所需的时间并不会很长。

运行下面的单元格，然后当它达到 `ACTIVE` 状态时继续。


In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

## 选择配方

在 Personalize 中，可用的算法被称为配方，下面的代码将给您提供一个完整的列表。它们各自有不同的使用案例，详细信息可在此处找到：https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html

在本次研讨会中，您将使用 HRNN（分层递归神经网络）引用文档：

```
HRRN 是一种分层递归神经网络，可以对给定时间范围内的用户-项目交互进行建模。当用户行为随时间变化时使用 HRNN 配方，这被称为演变意图问题。

为了训练模型，HRNN 使用来自数据集组的交互数据集。数据集组是一组相关的数据集，可以包括用户、项目和交互数据集。
```

以下文章详细地解释了数据集组与推荐的关系：https://openreview.net/pdf?id=ByzxsrrkJ4 


它在这里被用作使用深度神经网络构建的推荐系统的示例，并且可以仅使用我们的交互数据进行训练。当您稍后运行自己的实验时，这也是一个很好的起点。

In [ ]:
personalize.list_recipes()

下面的单元格选择 `HRNN` 配方。

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## 训练解决方案

在 Amazon Personalize 中，根据客户数据训练的模型称为解决方案，这些解决方案都是版本化的，因此您将首先创建一个解决方案，然后创建一个解决方案版本。这些版本用于根据较新的可用数据，跟踪模型随时间而发生的改进。

创建解决方案本身几乎是即时的，但是创建版本的实际训练可能需要一些时间。这通常是流程中等待时间最长的环节。如果您是在研讨会之外做这件事，您可以利用这段时间去查看电子邮件、喝杯咖啡等。

#### 创建解决方案

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-ri-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### 创建解决方案版本

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### 创建解决方案版本循环

训练模型可能需要一点时间，这通常需要至少 20 分钟才能完成。运行下面的单元格，再次等待它达到 `ACTIVE` 状态，然后再移动到下一位。

In [ ]:
current_time = datetime.datetime.now()
print("Training Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Training Completed on: ", current_time.strftime("%I:%M:%S %p"))

#### 获取解决方案版本的指标

既然您的解决方案和版本已经存在，您就可以获取指标来判断性能。

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## 创建并等待活动

现在您有了一个有效的解决方案版本，您需要创建一个活动，以便用于您的应用程序。活动只是模型的托管副本。同样会有一个短暂的等待，因此在执行之后，您可以在配置基础结构时稍微休息一下。

#### 创建活动

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ri-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### 等待活动状态变为 ACTIVE

#### 创建活动循环

在本节中，Personalize 将部署您的模型，这需要几分钟才能完成。执行下面的单元格以再次轮询，直到任务完成。应该需要 10 到 15 分钟。在单元格达到 `ACTIVE` 状态之后，继续。

In [ ]:
current_time = datetime.datetime.now()
print("Deploying Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Deploying Completed on: ", current_time.strftime("%I:%M:%S %p"))

## 获取样本推荐

活动状态变为活跃后，您就可以获得推荐。首先，我们需要从集合中选择一个随机用户。然后，我们将创建一些辅助函数，以获取电影信息以显示推荐，而不是仅显示 ID。

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']

#### 调用 GetRecommendations

下面的代码将获取针对随机选择的用户的推荐，然后将推荐放在数据架中并呈现它。

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Original Recommendations'])
recommendations_df

## 创建事件跟踪器

在您的推荐系统能够响应实时事件之前，您将需要一个事件跟踪器，以下代码将生成一个事件跟踪器，并可用于此实验室。您可以随意为之采用更巧妙的名称。

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTrackerRI',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## 模拟用户行为

下面的代码行提供了模拟用户与特定项目交互的代码示例，然后您将获得不同于开始时的推荐。

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the title of the movie for the header of the column
    movie_title_clicked = get_movie_title(movie_to_click)
    # Interact with the movie
    send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
    # Sleep for 2 seconds
    time.sleep(2)
    # Get new recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        title = get_movie_title(item['itemId'])
        recommendation_list.append(title)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

下面的 3 个单元格将模拟向数据集添加电影，然后在每次交互后呈现结果。第一列是最初的推荐，随后的列是您的用户现在已经与之交互的每部电影。列标题是用户与之交互的电影的名称，然后您可以看到推荐是如何变化的。

In [ ]:
movie_to_click = 180
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 210
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 415
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

## 结论

在本次研讨会中，您成功地根据交互创建了一个数据集组、一个数据集，根据数据训练了一个推荐模型，部署了一个活动来生成推荐，评估了最初的推荐，利用实时事件跟踪获得更好的推荐，下面是展示如何导出推荐的额外材料：

这些内容将在 GitHub 上公开，并可在您的组织内使用。为了构建自定义推荐模型，请查看未来的各种其他笔记本。

谢谢，祝您在下一个机器学习项目中好运！

额外材料如下：

## 批量推荐

到目前为止，您已经了解了如何通过 API 调用生成推荐，以及如何与事件跟踪器交互。这对于许多应用都很有效，但您可能会发现自己希望在本地缓存针对所有用户的推荐，甚至通过研究推荐来获得新的想法。为此，Amazon Personalize 还支持将您的推荐批量导出到文件中。下面的单元格将引导您把推荐发送到 S3 中的文件，然后显示推荐内容。该文件也可以从 S3 下载到本地计算机。

首先，您需要创建包含用户 ID 的 JSON 文件

In [ ]:
user_IDs = ['561', '233', '579']

json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in user_IDs:
        json_input.write('{"userId": "' + user_id + '"}\n')

现在将此文件上载到 S3：

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

输入文件在 S3 中可用之后，您需要定义输出的位置，并创建一个批量推理作业。

In [ ]:
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

下面的单元格创建批量作业。

In [ ]:
personalize_rec = boto3.client(service_name='personalize')
batchInferenceJobArn = personalize_rec.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "RI-Workshop-Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

下一个单元格将进行轮询，直到导出完成。这是研讨会的最后一个等待循环。这将再次轮询，直到它达到 `ACTIVE` 状态，在达到这个状态之后，您可以继续。

In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize_rec.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

成功导出数据后，获取文件并解析它。

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the movies
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

在上面的单元格中，您可以看到为用户提供的各种推荐，在真实场景中，用户列表可以是您的整个用户库，允许您快速参考和比较用户之间的结果。

## 额外材料：清理

如果您使用的是事件引擎，则下面的单元格完全是可选的，因为您的账户在研讨会后被删除。如果您想稍后在自己的账户中使用它，下面的单元格将删除所创建的资源以防止继续产生费用。

In [ ]:
# Delete the campaign:
personalize.delete_campaign(campaignArn=campaign_arn)
time.sleep(60)

In [ ]:
# Delete the solution
personalize.delete_solution(solutionArn=solution_arn)
time.sleep(60)

In [ ]:
# Delete the event tracker
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
time.sleep(60)

In [ ]:
# Delete the interaction dataset
personalize.delete_dataset(datasetArn=dataset_arn)
time.sleep(60)

In [ ]:
# Delete the event dataset
event_interactions_dataset_arn = dataset_arn
event_interactions_dataset_arn = event_interactions_dataset_arn.replace("INTERACTIONS", "EVENT_INTERACTIONS")
personalize.delete_dataset(datasetArn=event_interactions_dataset_arn)
time.sleep(60)

In [ ]:
# Delete the schema
personalize.delete_schema(schemaArn=schema_arn)

In [ ]:
# Delete the Dataset Group
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

In [ ]:
# Empty the S3 Bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.all().delete()
time.sleep(60)

In [ ]:
# Delete the S3 Bucket
bucket.delete()

In [ ]:
# IAM policies should also be removed
iam = boto3.client("iam")
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess", RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess",RoleName=role_name)

iam.delete_role(RoleName=role_name)

## 最后一步

清理完所有资源后，您现在可以关闭此窗口并返回到您开启的 GitHub 页面。README 的底部是删除您之前创建的 CloudFormation 堆栈的步骤。完成后，您就 100% 完成了研讨会。

祝贺您！